In [1]:
!pip install transformers datasets evaluate accelerate

from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer

from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00


2024-04-14 23:51:58.064899: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 23:51:58.065050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 23:51:58.222532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset = load_dataset("dair-ai/emotion", trust_remote_code=True)

train_data = dataset["train"]
test_data = dataset["test"]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_emotions_train = train_data.map(preprocess_function, batched=True)
tokenized_emotions_test = test_data.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [5]:
id2label = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}

label2id = {"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5}


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
num_c = 0

out = classifier(test_data['text'])
num_tot = len(out)

for i in range(len(out)):
    pred_label = label2id[out[i]['label']]
    if i<4:
        print("I: ", i,":: ", out[i])
        print("TD: ", test_data[i]['label'])
    if pred_label == test_data[i]['label']:
        num_c+=1

print("Number of Data Points: ", num_tot)
print("Accuracy for model without fine tuning: ", num_c/num_tot)

I:  0 ::  {'label': 'fear', 'score': 0.1746107041835785}
TD:  0
I:  1 ::  {'label': 'sadness', 'score': 0.18144166469573975}
TD:  0
I:  2 ::  {'label': 'anger', 'score': 0.18006038665771484}
TD:  0
I:  3 ::  {'label': 'anger', 'score': 0.1744869351387024}
TD:  1
Number of Data Points:  2000
Accuracy for model without fine tuning:  0.218


In [7]:


# model = AutoModelForMaskedLM.from_pretrained("distilbert/distilbert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id)

training_args = TrainingArguments(
    output_dir="llm_model_1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_emotions_train,
    eval_dataset=tokenized_emotions_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.248600,0.218995,0.923000
2,0.147200,0.174766,0.927500
3,0.098000,0.175279,0.923500


TrainOutput(global_step=3000, training_loss=0.24190740585327147, metrics={'train_runtime': 6459.9031, 'train_samples_per_second': 7.43, 'train_steps_per_second': 0.464, 'total_flos': 584777647046016.0, 'train_loss': 0.24190740585327147, 'epoch': 3.0})

In [8]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
num_c = 0

out = classifier(test_data['text'])
num_tot = len(out)

for i in range(len(out)):
    pred_label = label2id[out[i]['label']]
    if i<4:
        print("I: ", i,":: ", out[i])
        print("TD: ", test_data[i]['label'])
    if pred_label == test_data[i]['label']:
        num_c+=1

print("Number of Data Points: ", num_tot)
print("Accuracy for model with fine tuning: ", num_c/num_tot)

I:  0 ::  {'label': 'sadness', 'score': 0.9982592463493347}
TD:  0
I:  1 ::  {'label': 'sadness', 'score': 0.9986729621887207}
TD:  0
I:  2 ::  {'label': 'sadness', 'score': 0.9986799359321594}
TD:  0
I:  3 ::  {'label': 'joy', 'score': 0.9987579584121704}
TD:  1
Number of Data Points:  2000
Accuracy for model with fine tuning:  0.9275
